# Restaurant Agent Simulator-Based Evaluation

This notebook evaluates the Scheibmeir's restaurant agent using Azure AI Evaluation's simulator capabilities to generate contextually relevant test queries automatically.
Instead of using pre-defined queries, it uses AI to simulate realistic customer interactions.

## Setup and Configuration

In [ ]:
import os
import json
import asyncio
from dotenv import load_dotenv
from azure.ai.evaluation.simulator import Simulator
from azure.ai.evaluation import AzureOpenAIModelConfiguration
from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential

# Load environment variables from .env file
load_dotenv()

print("Libraries imported successfully!")

In [ ]:
# Configuration from environment variables - using RESTAURANT settings only
RESTAURANT_ASSISTANT_ID = os.getenv("RESTAURANT_ASSISTANT_ID")
RESTAURANT_ASSISTANT_MODEL = os.getenv("RESTAURANT_ASSISTANT_MODEL")
RESTAURANT_ASSISTANT_PROJECT = os.getenv("RESTAURANT_ASSISTANT_PROJECT")

print(f"Restaurant Assistant ID: {RESTAURANT_ASSISTANT_ID}")
print(f"Restaurant Assistant Model: {RESTAURANT_ASSISTANT_MODEL}")
print(f"Restaurant Project: {RESTAURANT_ASSISTANT_PROJECT}")

## Initialize Clients

In [ ]:
# Initialize Azure credential and agents client
credential = DefaultAzureCredential()
agents_client = AgentsClient(
    endpoint=RESTAURANT_ASSISTANT_PROJECT,
    credential=credential
)

print("Clients initialized successfully!")

## Define Target Function for Simulator

This async function will be called by the simulator to test different scenarios.

In [ ]:
async def query_restaurant_agent_async(query: str) -> dict:
    """
    Async function to query the restaurant agent.
    Used by the simulator for dynamic testing.
    """
    try:
        # Create a thread and run with the agent
        result = agents_client.create_thread_and_process_run(
            agent_id=RESTAURANT_ASSISTANT_ID,
            thread={
                "messages": [
                    {
                        "role": "user",
                        "content": query
                    }
                ]
            }
        )
        
        if result.status.value == "completed":
            # Get the agent's response
            messages = agents_client.messages.list(thread_id=result.thread_id)
            for msg in messages:
                if msg.role.value == "assistant":
                    return {
                        "response": msg.content[0].text.value,
                        "query": query
                    }
        
        return {
            "response": f"Agent run failed with status: {result.status.value}",
            "query": query
        }
        
    except Exception as e:
        return {
            "response": f"Error querying agent: {str(e)}",
            "query": query
        }

# Test the function
test_result = await query_restaurant_agent_async("What are the hours for Scheibmeir's?")
print("Test query result:")
print(f"Query: {test_result['query']}")
print(f"Response: {test_result['response'][:100]}...")

## Simulate Customer Scenarios

Note: For this demo, we'll create manual simulation scenarios instead of using the full Azure AI Simulator which requires additional configuration.

In [ ]:
# Define different customer personas and scenarios
simulation_scenarios = [
    {
        "persona": "First-time customer",
        "queries": [
            "What type of restaurant is Scheibmeir's?",
            "What are your most popular dishes?",
            "Do you take reservations?"
        ]
    },
    {
        "persona": "Dietary restrictions customer", 
        "queries": [
            "Do you have vegetarian options?",
            "What gluten-free dishes do you offer?",
            "Can you accommodate food allergies?"
        ]
    },
    {
        "persona": "Location-focused customer",
        "queries": [
            "Where is Scheibmeir's located?",
            "What are your hours?",
            "Is there parking available?"
        ]
    },
    {
        "persona": "Price-conscious customer",
        "queries": [
            "What are your prices like?",
            "Do you have any specials?",
            "What's the average cost for dinner?"
        ]
    }
]

print(f"Defined {len(simulation_scenarios)} customer personas with simulation scenarios")

## Run Simulation

In [ ]:
# Run simulation for each persona
print("Running Customer Simulation:")
print("=" * 60)

all_results = []

for scenario in simulation_scenarios:
    persona = scenario["persona"]
    queries = scenario["queries"]
    
    print(f"\n🎭 Testing {persona}:")
    print("-" * 40)
    
    persona_results = []
    
    for query in queries:
        print(f"\n❓ Query: {query}")
        
        # Test the query
        result = await query_restaurant_agent_async(query)
        persona_results.append(result)
        
        # Display response
        response = result["response"]
        if len(response) > 150:
            display_response = response[:150] + "..."
        else:
            display_response = response
            
        print(f"💬 Response: {display_response}")
    
    all_results.append({
        "persona": persona,
        "results": persona_results
    })

print(f"\n✅ Simulation completed for {len(simulation_scenarios)} personas!")

## Analyze Simulation Results

In [ ]:
# Analyze the simulation results
print("Simulation Analysis:")
print("=" * 50)

total_queries = 0
successful_responses = 0
restaurant_focused_responses = 0

for persona_data in all_results:
    persona = persona_data["persona"]
    results = persona_data["results"]
    
    print(f"\n{persona}:")
    
    persona_success = 0
    persona_restaurant_focus = 0
    
    for result in results:
        total_queries += 1
        response = result["response"].lower()
        
        # Check if response is successful (not an error)
        if not response.startswith("error") and not response.startswith("agent run failed"):
            successful_responses += 1
            persona_success += 1
            
            # Check if response mentions restaurant-specific content
            if "scheibmeir" in response or "restaurant" in response or "menu" in response:
                restaurant_focused_responses += 1
                persona_restaurant_focus += 1
    
    print(f"  ✓ Success rate: {(persona_success/len(results))*100:.1f}%")
    print(f"  🎯 Restaurant focus: {(persona_restaurant_focus/len(results))*100:.1f}%")

print(f"\nOverall Simulation Results:")
print(f"Total queries tested: {total_queries}")
print(f"Successful responses: {successful_responses}")
print(f"Overall success rate: {(successful_responses/total_queries)*100:.1f}%")
print(f"Restaurant-focused responses: {restaurant_focused_responses}")
print(f"Restaurant focus rate: {(restaurant_focused_responses/total_queries)*100:.1f}%")

print("\n" + "=" * 50)
print("🎉 Simulator-based evaluation completed!")
print("\nThis simulation tested how well the agent handles different")
print("customer personas and their typical questions about the restaurant.")